In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

import src.dataset_manager as dataset_manager

In [3]:
dataset_manager.DatasetManager.list_datasets_by_group().keys()

dict_keys(['geometry_of_truth', 'relations', 'sst2', 'md_gender', 'snli', 'ag_news', 'ner', 'tense', 'language_identification', 'singular_plural'])

In [4]:
geometry_of_truth_dataset = dataset_manager.DatasetManager.from_dataset_group("geometry_of_truth", batch_size=2)

In [5]:
len(geometry_of_truth_dataset)

6478

In [ ]:
batch = next(iter(geometry_of_truth_dataset))
batch

[ContextQASample(context='The city of Yaroslavl is in Russia.', questions=['# Do the facts support this statement?', '# Do you find this to be truthful?', '# May I assume this is correct?', '# Are these facts correct?', '# Do you think this checks out?', '# Is this true?', '# Should I believe this statement?', '# Can you confirm the accuracy of this?', '# Does this information ring true?', '# Does this statement hold true?'], answers=['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes']),
 ContextQASample(context='King Henry VIIIs only surviving legitimate child to survive childhood was Queen Elizabeth.', questions=['# Are you saying this is correct?', '# Can we confirm this is accurate?', '# Has this been verified as true?', '# Would you say this is accurate?', '# Could you tell me if this is factual?', '# Have I understood this correctly?', '# Are these facts correct?', '# Can you confirm the accuracy of this?', '# Is this statement true?', '# Am I right to believe t

In [7]:
train, test = geometry_of_truth_dataset.split([0.7, 0.3])

In [ ]:
print(len(train), len(test))

4535 1943


In [9]:
named_dataset = dataset_manager.DatasetManager.from_named_datasets([("geometry_of_truth", "cities"), ("geometry_of_truth", "neg_cities")], batch_size=4)

In [10]:
batch = next(iter(named_dataset))
batch

[ContextQASample(context='The city of Kaduna is in Nigeria.', questions=['# Can you verify if this is true?', '# Would you consider this to be true?', '# Is this true?', '# Would you say this is accurate?', '# Does this information ring true?', '# Could you tell me if this is factual?', '# Should I believe this statement?', '# May I assume this is correct?', '# Are these facts correct?', '# Has this been verified as true?'], answers=['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes']),
 ContextQASample(context='The city of San Miguel de Tucuman is in Argentina.', questions=['# Can we confirm this is accurate?', '# May I assume this is correct?', '# Are you saying this is correct?', '# Would you consider this to be true?', '# Are these facts correct?', '# Can you verify if this is true?', '# Has this been verified as true?', '# Am I right to believe this?', '# Can you confirm the accuracy of this?', '# Do the facts support this statement?'], answers=['Yes', 'Yes', 'Ye

In [11]:
relation_dataset = dataset_manager.DatasetManager.from_dataset_group(
    "relations",
    batch_size=4
)

In [12]:
next(iter(relation_dataset))

[ContextQASample(context='Abu Ghosh is in the country of Israel.', questions=['# May I assume this is correct?', '# Has this been verified as true?', '# Is this statement true?', '# Does this information ring true?', '# Is this true?', '# Can you verify if this is true?', '# Could you tell me if this is factual?', '# Do you think this checks out?', '# Should I believe this statement?', '# Do the facts support this statement?'], answers=['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes']),
 ContextQASample(context='Selim Bouadla plays in the position of a midfielder.', questions=['# Would you validate this statement?', '# Has this been verified as true?', '# Do you find this to be truthful?', '# Can you verify if this is true?', '# Are you confirming this is true?', '# Would you consider this to be true?', '# Do you agree that this is true?', '# Are you saying this is correct?', '# Should I believe this statement?', '# Can we confirm this is accurate?'], answers=['Yes

In [13]:
sst2_dataset = dataset_manager.DatasetManager.from_dataset_group("sst2", batch_size=4)

In [14]:
next(iter(sst2_dataset))

[ContextQASample(context='sticks rigidly to the paradigm ,', questions=['# Is the general sentiment of this review positive?', '# Is the sentiment in this review on the positive side?', '# This review is negative. Do you agree?', '# Would you say this is an unfavorable review?', '# Does this review indicate a good experience?', '# Does this review lean towards the negative end of the spectrum?', '# Based on the language used, is this a negative review?', '# Are the comments in this review predominantly negative?', '# Are the comments in this review predominantly positive?', '# Would you say this is a favorable review?'], answers=['No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No']),
 ContextQASample(context='keep you watching ,', questions=['# Is the general sentiment of this review negative?', '# This review is positive. Do you agree?', '# Can we consider this a thumbs-down review?', '# Does this review indicate a good experience?', '# Is the general sentiment of this rev

In [15]:
gender_dataset = dataset_manager.DatasetManager.from_dataset_group("md_gender", batch_size=4)

In [16]:
next(iter(gender_dataset))

[ContextQASample(context="It's important to take a deep breath and focus when piloting because every second counts, the Superbird-B1 is no exception.\n\nThis text is about Superbird-B1.", questions=['# Am I learning about a female individual here?', '# Am I reading about a male person here?', '# Am I reading about someone who identifies as female?', '# Does this text feature a female person?', '# Is this talking about a female person?', '# Is the character described here male?', '# Does this portray a male individual?', '# Is this narrative about a female individual?', '# Does this refer to a male individual?', '# Is the focus of this sentence on someone male?'], answers=['No', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes']),
 ContextQASample(context='Davide Plebani made Italy proud when he competed at the 2016 UEC European Track Championships in the elimination race event.\n\nThis text is about Davide Plebani.', questions=['# Am I reading about someone who identifies as ma

In [ ]:
len(gender_dataset)

2229

In [18]:
news_dataset = dataset_manager.DatasetManager.from_dataset_group("ag_news", batch_size=4)

In [ ]:
next(iter(news_dataset))

[ContextQASample(context='Hendrick Motorsports plane crashes, killing 10 on way to NASCAR &lt;b&gt;...&lt;/b&gt;\n\nA Hendrick Motorsports plane crashed Sunday on its way to a NASCAR race, killing all 10 people aboard, including the son, brother and two nieces of the owner of one of auto racing #39;s most successful organizations.', questions=['# Can this be considered a Sports-related article?', '# In terms of content, does this piece deal with World News?', '# Does this article primarily address Science/Technology?', '# Can this article be categorized under World News?', '# In essence, is this a piece about Sports?', '# Is Sports a key subject in this piece?', '# To what degree does this article cover World News?', '# Does the scope of this article encompass Business?', '# Is there a substantial focus on World News in this article?', '# Does this writing concentrate on aspects of Sports?'], answers=['Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes']),
 ContextQASample(co

In [20]:
ner_dataset = dataset_manager.DatasetManager.from_dataset_group("ner", batch_size=4)

Reading NER dataset
Processing NER dataset


100%|██████████████████████████████████████████████████████████████████████████| 47959/47959 [00:21<00:00, 2266.67it/s]


In [21]:
next(iter(ner_dataset))

[ContextQASample(context='Iran insists its nuclear program is used for the peaceful purpose of energy generation and has said it will soon resume uranium processing .', questions=['# Does this passage make any reference to Malay Peninsula?', '# Can you find any references to Port Elizabeth in this text?', '# Does this text contain the term Iran anywhere?', '# Are there any occurrences of Sudan Said Djinnit within this content?', '# Does this document contain any mentions of Iran?', '# Does this text mention Iran?', '# Is there any mention of Iran throughout this material?', '# Does this text have any instances of Iran?', '# Is Project Venezuela mentioned anywhere in the passage?', '# Is Dodger Stadium noted anywhere within these paragraphs?'], answers=['No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No']),
 ContextQASample(context='At least five people have died from bird flu in China this year .', questions=['# Does this text contain the term Kilju anywhere?', '# Is China 

In [22]:
snli_dataset = dataset_manager.DatasetManager.from_dataset_group("snli", batch_size=4)

Loading SNLI dataset


100%|███████████████████████████████████████████████████████████████████████| 550152/550152 [00:26<00:00, 20392.10it/s]


In [23]:
next(iter(snli_dataset))

[ContextQASample(context='A soccer game, with a huge crowd in the background.', questions=['# Would you say these two sentences connect logically? People playing soccer in front of a crowd.', '# Is there logical continuity between these sentences? People playing soccer in front of a crowd.', '# After reading the first sentence, is the second one coherent? People playing soccer in front of a crowd.', '# Would the following sentence align with what we just read? People playing soccer in front of a crowd.', '# Does this next sentence fit with the preceding context? People playing soccer in front of a crowd.', '# Looking at the previous context, does this next part hold up? People playing soccer in front of a crowd.', '# Following that statement, does this next part flow naturally? People playing soccer in front of a crowd.', '# Building on that last sentence, does this next one make sense? People playing soccer in front of a crowd.', '# Based on what was just stated, is this a reasonable 

In [24]:
tense_dataset = dataset_manager.DatasetManager.from_dataset_group("tense", batch_size=4)

In [25]:
next(iter(tense_dataset))

[ContextQASample(context="In a week's time, she will have been working on her thesis for a year.", questions=['# Would you identify this as an example of the present tense?', '# Is the time frame of this statement future?', '# Is the narrative presented in the present tense?', '# Does this sentence convey the past tense?', '# Am I correct in saying this is in the present tense?', '# Is the narrative presented in the past tense?', '# This statement is in the present tense. Do you agree?', '# Can we classify this sentence as being in the future tense?', '# Can we classify this sentence as being in the future tense?', '# Would you say this is written in the future tense?'], answers=['No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes']),
 ContextQASample(context='She has rescued a stray dog.', questions=['# Would you identify this as an example of the future tense?', '# Would you say this is written in the future tense?', '# Is the structure of this sentence indicative of the fu

In [26]:
len(tense_dataset)

1687

In [27]:
language_dataset = dataset_manager.DatasetManager.from_dataset_group("language_identification", batch_size=4)

README.md:   0%|          | 0.00/553 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/22000 [00:00<?, ? examples/s]

In [28]:
next(iter(language_dataset))

[ContextQASample(context='moyle p y j cech fishes an introduction to ichthyology a edición upper saddle river nueva jersey estados unidos prentice-hall año ', questions=['# Am I correct in saying this is in Spanish?', '# Is the text presented in Urdu?', '# Am I correct in saying this is in Spanish?', '# Would you agree that this is in Thai?', '# Would you classify this as written in Spanish?', '# Can we confirm this is in Spanish?', '# Is the language used here Spanish?', '# Can we determine that this is in Turkish?', '# Would you agree that this is in Romanian?', '# Does this passage use Hindi?'], answers=['Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No']),
 ContextQASample(context='ในปี คศ  ไบเยวูและทัคเกอร์กลับมารวมตัวกันอีกครั้ง และออกทัวร์คอนเสิร์ตในสหราชอาณาจักรและไอร์แลนด์ ในเดือนกุมภาพันธ์และมีนาคม คศ ', questions=['# Is the text presented in Korean?', '# Is the text presented in Turkish?', '# Is the language of this text Thai?', '# Would you agree that this is i

In [29]:
singular_plural_dataset = dataset_manager.DatasetManager.from_dataset_group("singular_plural", batch_size=4)

In [30]:
next(iter(singular_plural_dataset))

[ContextQASample(context='The drivers transported passengers.', questions=['# Is this statement about one entity?', '# Can we interpret this as referring to a single entity?', '# Is the reference here to more than one individual?', '# Is this statement about one entity?', '# Is the reference here to one individual?', '# Does this sentence focus on a single person?', '# Is this text discussing one individual?', '# Is the reference here to one individual?', '# Is the reference here to more than one individual?', '# Is this statement about one entity?'], answers=['No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No']),
 ContextQASample(context='Keith writes songs.', questions=['# Does this statement indicate a plural subject?', '# Is the reference here to more than one individual?', '# Does this sentence focus on a single person?', '# Is this text discussing one individual?', '# Is this sentence referring to one individual?', '# Is this statement about more than one entity?', '# Is

In [31]:
len(singular_plural_dataset)

92